**SageMaker Repo:  Workflow/Airflow**:  
https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/workflow/airflow.py

**Airflow Repo:  SageMaker Operators**: 
https://github.com/apache/airflow/tree/master/airflow/providers/amazon/aws

# Define the Directed Acyclic Graph (DAG) of Execution

In [ ]:
import airflow
from airflow import DAG

default_args = {
    'owner': 'airflow',
    'provide_context': True
}

dag = DAG('bert_reviews', default_args=default_args,
          schedule_interval='@once')

# SageMaker Processing Job Operator

In [ ]:
from airflow.contrib.operators.sagemaker_processing_operator import SageMakerProcessingOperator
from sagemaker.workflow.airflow import processing_config

process_config = process_config(estimator=estimator,
                                inputs=your_training_data_s3_uri)

process_op = SageMakerProcessingOperator(
    task_id='process',
    config=processing_config,
    wait_for_completion=True,
    dag=dag)

# SageMaker Training Job Operator

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_train.py',
#                       role='sagemaker-role',
#                       framework_version='1.11.0',
#                       training_steps=1000,
#                       evaluation_steps=100,
#                       instance_count=2,
#                       instance_type='ml.p2.xlarge')



In [ ]:
from airflow.contrib.operators.sagemaker_training_operator import SageMakerTrainingOperator
from sagemaker.workflow.airflow import training_config

train_config = training_config(estimator=estimator,
                               inputs=your_training_data_s3_uri)

train_op = SageMakerTrainingOperator(
    task_id='train',
    config=train_config,
    wait_for_completion=True,
    dag=dag)

train_op.set_upstream(process_op)

# SageMaker Model Operator

In [ ]:
from airflow.contrib.operators.sagemaker_model_operator import SageMakerModelOperator
from sagemaker.workflow.airflow import model_config

model_op = SageMakerModelOperator(
    task_id='model',
    config=model_config,
    wait_for_completion=True,
    dag=dag)

model_op.set_upstream(train_op)

# SageMaker Endpoint Operator

In [ ]:
from airflow.contrib.operators.sagemaker_endpoint_operator import SageMakerEndpointOperator
from sagemaker.workflow.airflow import endpoint_config, # deploy_config_from_estimator

deploy_op = SageMakerEndpointOperator(
    task_id='deploy',
    config=endpoint_config,
    wait_for_completion=True,
    dag=dag)

deploy_op.set_upstream(model_op)